The hardest part was figuring out what the program
was supposed to accomplish.

From the email, code documentation, and code itself,
I had a very difficult time understanding what the code
was trying to accomplish, nevermind how the code was 
accomplishing it.

After studying the code, I came with my description
of what the code is to accomplish. This was 
corroborated by the example output.

    for number of months from 0 to 11 inclusive,
    calculate the maximum possible number of days
    in (number of months + 1) consecutive months
    have. (Choose the starting month that gives highest possible answer.)
    in starting_month and the next
    
n_months months for all possible starting_months

Later on, I figured out the first column in the sample
output was the earliest or latest starting month in a year
that would produce the max number of days for n months.

For all my code below, months are numbered starting at 0.
That is,

0 means January

11 means December

In [1]:
MONTHS_PER_YEAR = 12

In [2]:
# for unknown year
def max_month_length(month):
    max_month_lengths = (
        31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31)
    return max_month_lengths[month % len(max_month_lengths)]

In [3]:
max_month_length(0)

31

In [4]:
max_month_length(13)

29

In [5]:
def max_days_n_months(n_months, starting_month=None):
    if starting_month is not None:
        return sum(
            max_month_length(month)
            for month in range(starting_month, starting_month + n_months + 1)
        )
    
    return max(
        max_days_n_months(n_months, starting_month)
        for starting_month in range(MONTHS_PER_YEAR)
    )

In [6]:
def foo(n):
    for n_months in range(n):
        n_days = max_days_n_months(n_months)
        yield n_months, n_days

In [7]:
n = MONTHS_PER_YEAR
# %timeit list(foo(n))
list(foo(n))

[(0, 31),
 (1, 62),
 (2, 92),
 (3, 123),
 (4, 153),
 (5, 184),
 (6, 215),
 (7, 245),
 (8, 276),
 (9, 306),
 (10, 337),
 (11, 366)]

That has the correct maximum number of days.

Now to add the stuff that keeps track of which starting months can yield those 

In [8]:
from collections import defaultdict

def foo(n):
    for n_months in range(n):
        d = defaultdict(list)
        for starting_month in range(MONTHS_PER_YEAR):
            n_days = max_days_n_months(n_months, starting_month)
            d[n_days].append(starting_month)
        max_n_days = max(d.keys())
        yield n_months, max_n_days, sorted(d[max_n_days])

In [9]:
n = MONTHS_PER_YEAR
# %timeit list(foo(n))
list(foo(n))

[(0, 31, [0, 2, 4, 6, 7, 9, 11]),
 (1, 62, [6, 11]),
 (2, 92, [2, 4, 5, 6, 7, 9, 10]),
 (3, 123, [4, 6, 9]),
 (4, 153, [2, 3, 4, 5, 6, 7, 8]),
 (5, 184, [2, 4, 6, 7]),
 (6, 215, [6]),
 (7, 245, [2, 4, 5]),
 (8, 276, [4]),
 (9, 306, [2, 3]),
 (10, 337, [2]),
 (11, 366, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])]

In [10]:
def pretty(n):
    x = tuple(foo(n))
    for f, s in ((min, '-gt'), (max, '-ge')):
        print(f'Compare: {s} ')
        for n_months, max_n_days, months in foo(n):
            if n_months == 0:
                print(f'month: {f(months)+1} month spans: {max_n_days} days  ')
            elif n_months == 1:
                print(f'month: {f(months)+1} plus following month spans: {max_n_days} days  ')
            else:
                print(f'month: {f(months)+1} plus following {n_months} months spans: {max_n_days} days  ')
        print()
    

In [11]:
n = MONTHS_PER_YEAR
pretty(n)

Compare: -gt 
month: 1 month spans: 31 days  
month: 7 plus following month spans: 62 days  
month: 3 plus following 2 months spans: 92 days  
month: 5 plus following 3 months spans: 123 days  
month: 3 plus following 4 months spans: 153 days  
month: 3 plus following 5 months spans: 184 days  
month: 7 plus following 6 months spans: 215 days  
month: 3 plus following 7 months spans: 245 days  
month: 5 plus following 8 months spans: 276 days  
month: 3 plus following 9 months spans: 306 days  
month: 3 plus following 10 months spans: 337 days  
month: 1 plus following 11 months spans: 366 days  

Compare: -ge 
month: 12 month spans: 31 days  
month: 12 plus following month spans: 62 days  
month: 11 plus following 2 months spans: 92 days  
month: 10 plus following 3 months spans: 123 days  
month: 9 plus following 4 months spans: 153 days  
month: 8 plus following 5 months spans: 184 days  
month: 7 plus following 6 months spans: 215 days  
month: 6 plus following 7 months spans: 245 

The above output matches days_spanned.sh.stdout, but is ugly.
The code above is not as readable as I like.

I hope to improve both tomorrow.